# 09 — Run All Experiments (E1–E5)

Este notebook ejecuta el paquete completo de experimentos con todos los modelos y exógenas.

**Supuesto experimental (oracle exog):** todas las exógenas están disponibles durante el horizonte de predicción.


In [4]:
import sys; print(sys.executable)
import site; print(site.getsitepackages())
import torch; print(torch.__version__); print(torch.cuda.is_available())

/home/sagemaker-user/.venv/bin/python
['/home/sagemaker-user/.venv/lib/python3.11/site-packages']
2.5.1+cu124
True


In [5]:
# Imports y configuración base
from __future__ import annotations

import json
import sys
from pathlib import Path

import numpy as np
import pandas as pd

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.common import load_data, make_features
from src.experiments import (
    run_E1_walk_forward,
    run_E2_last_39_weeks,
    run_E3_loso,
    run_E4_train35_test10low,
    run_E5_unemployment_shock,
    collect_summary_metrics,
)

from src.models import (
    SarimaxForecaster,
    ProphetForecaster,
    DeepARForecaster,
    LSTMForecaster,
    TransformerForecaster,
)

SEED = 42
np.random.seed(SEED)

DATA_PATH = PROJECT_ROOT / 'data' / 'Walmart_Sales.csv'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Data path:', DATA_PATH)

Project root: /mnt/custom-file-systems/s3/shared/TFMAXEL
Data path: /mnt/custom-file-systems/s3/shared/TFMAXEL/data/Walmart_Sales.csv


## Configuración común
- Lags y rollings (solo pasado)
- Exógenas obligatorias
- Horizonte de test = últimas 39 semanas

In [6]:
config = {
    'seed': SEED,
    'lags': [1, 2, 4, 8, 52],
    'rollings': [4, 8, 12],
    'exog_cols': ['Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment'],
    'add_calendar': True,
    'lookback': 52,
    'test_weeks': 39,
    'output_dir': str(OUTPUTS_DIR),
    'epochs': 20,
    'batch_size': 64,
    'lr': 1e-3,
}

df = load_data(DATA_PATH)
df_feat, feature_cols = make_features(
    df,
    lags=config['lags'],
    rollings=config['rollings'],
    add_calendar=config['add_calendar'],
    group_col='Store',
)
config['feature_cols'] = feature_cols

# Nota: los modelos que usan lags/rollings calcularán features de forma recursiva en predicción
df_feat.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,lag_1,lag_2,...,lag_52,roll_mean_4,roll_std_4,roll_mean_8,roll_std_8,roll_mean_12,roll_std_12,weekofyear,month,year
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,2010
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,1643690.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,2010
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,1641957.44,1643690.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,2,2010
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,1611968.17,1641957.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,2,2010
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,1409727.59,1611968.17,...,NaN,1576836.025,112353.415114,NaN,NaN,NaN,NaN,9,3,2010


## Metadata de experimentos
Se guarda en `outputs/metadata_experiments.json` con fechas y versiones.

In [7]:
try:
    from importlib.metadata import version
except Exception:
    version = None

def safe_version(pkg: str) -> str | None:
    if version is None:
        return None
    try:
        return version(pkg)
    except Exception:
        return None

metadata_exp = {
    'seed': SEED,
    'exog_cols': config['exog_cols'],
    'lags': config['lags'],
    'rollings': config['rollings'],
    'lookback': config['lookback'],
    'test_weeks': config['test_weeks'],
    'libs': {
        'numpy': safe_version('numpy'),
        'pandas': safe_version('pandas'),
        'statsmodels': safe_version('statsmodels'),
        'prophet': safe_version('prophet'),
        'gluonts': safe_version('gluonts'),
        'torch': safe_version('torch'),
    },
    'oracle_exog_assumption': 'All exogenous covariates are assumed available over the forecast horizon (oracle scenario).',
}

meta_path = OUTPUTS_DIR / 'metadata_experiments.json'
meta_path.write_text(json.dumps(metadata_exp, indent=2, ensure_ascii=False), encoding='utf-8')
print('Wrote:', meta_path)

Wrote: /mnt/custom-file-systems/s3/shared/TFMAXEL/outputs/metadata_experiments.json


## Instancia de modelos
Si falta una dependencia (Prophet/GluonTS), el modelo se omite con un aviso.

In [8]:
models = []

# M1 SARIMAX
models.append(SarimaxForecaster())

# M2 Prophet
try:
    models.append(ProphetForecaster())
except Exception as exc:
    print('Skipping Prophet:', exc)

# M3 DeepAR
try:
    models.append(DeepARForecaster())
except Exception as exc:
    print('Skipping DeepAR:', exc)

# M4 LSTM
try:
    models.append(LSTMForecaster())
except Exception as exc:
    print('Skipping LSTM:', exc)

# M5 Transformer
try:
    models.append(TransformerForecaster())
except Exception as exc:
    print('Skipping Transformer:', exc)

models

[SarimaxForecaster(name='sarimax_exog', is_global=False),
 ProphetForecaster(name='prophet_regressors', is_global=False),
 DeepARForecaster(name='deepar_exog', is_global=True),
 LSTMForecaster(name='lstm_exog', is_global=True),
 TransformerForecaster(name='transformer_exog', is_global=True)]

## Ejecutar E1 y E2 (todos los modelos)

In [9]:
run_E1_walk_forward(models, df, config)
split_info = run_E2_last_39_weeks(models, df, config)
split_info

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type        | Params | Mode  | In sizes                                                     | Out sizes  
---------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 24.6 K | train | [[1, 1], [1, 1], [1, 208, 8], [1, 208], [1, 208], [1, 1, 8]] | [1, 100, 1]
---------------------------------------------------------------------------------------------------------------------------
24.6 K    Trainable params
0         Non-trainable params
24.6 K    Total params
0.098     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 10.77523 (best 10.77523), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_0/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 9.72098 (best 9.72098), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_0/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 9.69014 (best 9.69014), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_0/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 9.54870 (best 9.54870), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_0/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 9.46794 (best 9.46794), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 10.64424 (best 10.64424), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_1/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 9.74909 (best 9.74909), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_1/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 9.69704 (best 9.69704), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_1/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' reached 9.66450 (best 9.66450), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_1/checkpoints/epoch=3-step=200.ckpt' as top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 9.63882 (best 9.63882), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL

...

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 11.51593 (best 11.51593), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_37/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 10.55604 (best 10.55604), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_37/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' was not in top 1
INFO: Epoch 3, global step 200: 'train_loss' was not in top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 10.43288 (best 10.43288), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_37/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 10.36076 (best 10.36076), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_37/checkpoints/epoch=5-step=300.ckpt' as top 1
IN

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 11.83255 (best 11.83255), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_38/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 10.62778 (best 10.62778), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_38/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 10.58077 (best 10.58077), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_38/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO: Epoch 3, global step 200: 'train_loss' was not in top 1
INFO: Epoch 4, global step 250: 'train_loss' reached 10.46894 (best 10.46894), saving model to '/mnt/custom-file-systems/s3/shared/TFMAXEL/notebooks/lightning_logs/version_38/checkpoints/epoch=4-step=250.ckpt' as top 1
INFO: Epoch 5, global step 300: 'train_loss' reached 10.45022 (b

ValueError: Not enough data to build LSTM sequences (check lookback / feature NaNs).

## Ejecutar E3 y E4 (solo modelos globales)

In [ ]:
models_global = [m for m in models if getattr(m, 'is_global', False)]
run_E3_loso(models_global, df, config)
run_E4_train35_test10low(models_global, df, config)

## Ejecutar E5 (shock de desempleo +15%)

In [ ]:
run_E5_unemployment_shock(models, df, config, shock_pct=0.15)

## Resumen de métricas

In [ ]:
summary = collect_summary_metrics(OUTPUTS_DIR)
summary_path = OUTPUTS_DIR / 'experiments' / 'summary_metrics.csv'
summary_path.parent.mkdir(parents=True, exist_ok=True)
summary.to_csv(summary_path, index=False)
summary_path